In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=30, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")
n_pop = 10
n_gen = 10
n_sats = 12

In [3]:
# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
# poly = Create_Poly("../../Input_Files/Targets_File.txt")

# Writing random points within the polygon to a target file.
targets_filename = "../../Input_Files/Targets_File.txt"
# polygon_random_points(poly,100,targets_filename)

# Loading targets into stk from file.
stk_object.Target_Loader(targets_filename)

In [4]:
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [3]:

opt = Optimizer(stk_object,n_pop,n_gen,n_sats,weights=(7,-8,-1,-2))

In [4]:
stk_object.Interpolate=True
stk_object.dt = 30
starting_ind = get_ind()                                # Gets individual associating with ../../Input_Files/Satellite_File.txt
hof,percent,std,time,sats = opt.run(starting_ind)       # Run optimization putting individual into the initial population
opt.cost_function(hof[0])                               # Get best individual from optimization and load them into ../../Input_Files/Satellite_File.txt

In [5]:
stk_object.dt = 2.5
stk_object.Generate_Planning_Data()

- Computing_AER Data/Default [==========] 1200/1200 [100%] in 41:30.7 (0.48/s)  
- Computing_Sat Angles Data [==========] 1200/1200 [100%] in 16:25.5 (1.22/s)   [          ] ▁▃▅ 53/1200 [4%] in 42s (~15:12, 1.3/s)[>         ] ▆█▆ 118/1200 [10%] in 1:35 (~14:33, 1.2[>         ] ▆▄▂ 130/1200 [11%] in 1:44 (~14:18, 1.2[>         ] ▅▇▇ 135/1200 [11%] in 1:49 (~14:14, 1.2[=>        ] ▄▂▂ 188/1200 [16%] in 2:31 (~13:36, 1.2[=>        ] ▇▅▃ 212/1200 [18%] in 2:52 (~13:24, 1.2[=>        ] ▄▆█ 252/1200 [21%] in 3:24 (~12:46, 1.2[==>       ] ▅▃▁ 306/1200 [26%] in 4:09 (~12:08, 1.2[==>       ] ▅▃▁ 321/1200 [27%] in 4:20 (~11:55, 1.2[==>       ] ▇▅▃ 335/1200 [28%] in 4:31 (~11:39, 1.2[==>       ] ▆█▆ 351/1200 [29%] in 4:44 (~11:28, 1.2[==>       ] ▁▃▅ 375/1200 [31%] in 5:05 (~11:12, 1.2[==>       ] ▅▇▇ 379/1200 [32%] in 5:09 (~11:09, 1.2[===>      ] ▂▄▆ 441/1200 [37%] in 5:58 (~10:15, 1.2[===>      ] ▃▁▃ 444/1200 [37%] in 6:00 (~10:13, 1.2[===>      ] ▆▄▂ 474/1200 [40%] in 6:27 (~9:52, 1.2/[====> 

0

In [6]:
stk_object.Plan_Data(slew_rate=1)

- Planning_Data [==========! (!) 32065/32400 [99%] in 7:24:08.5 (1.20/s)        [>         ] ▁▃▅ 3728/32400 [12%] in 6s (~49s, 583.4/s) [>         ] ▅▇▇ 3765/32400 [12%] in 7s (~50s, 572.4/s) [=>        ] ▂▂▄ 4828/32400 [15%] in 13s (~1:13, 375.1/s) [=>        ] █▆▄ 5798/32400 [18%] in 21s (~1:37, 275.1/s) [==>       ] ▅▃▁ 9988/32400 [31%] in 1:39 (~3:43, 100.6/s) [==>       ] ▆▄▂ 10269/32400 [32%] in 1:48 (~3:52, 95.2/s) [===>      ] █▆▄ 10583/32400 [33%] in 1:58 (~4:04, 89.5/s) [===>      ] ▅▃▁ 12147/32400 [37%] in 3:02 (~5:03, 66.7/s) [===>      ] ▃▅▇ 13124/32400 [41%] in 3:55 (~5:45, 55.9/s) [====>     ] ▂▂▄ 14519/32400 [45%] in 5:31 (~6:48, 43.8/s) [====>     ] ▄▂▂ 14869/32400 [46%] in 6:00 (~7:04, 41.3/s) [====>     ] ▅▇▇ 15445/32400 [48%] in 6:53 (~7:33, 37.4/s) [====>     ] ▂▂▄ 16090/32400 [50%] in 8:00 (~8:06, 33.5/s) [======>   ] ▂▄▆ 21089/32400 [65%] in 25:10 (~13:30, 14.0/s) [======>   ] ▇▇▅ 21350/32400 [66%] in 26:46 (~13:51, 13.3/s) [======>   ] ▇▅▃ 21430/32400 [66%] in 2

KeyboardInterrupt: 

In [8]:
data_comparison = {}
data_comparison["Unplanned (%)"] = [len(np.unique(stk_object.Planning_Data[stk_object.Planning_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = [max(stk_object.Planning_Data[stk_object.Planning_Data['Target'].values==tar_num]['Time'].values/86400) for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values/86400) for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

    Unplanned (%)
0           100.0
1           100.0
2           100.0
3           100.0
4           100.0
..            ...
95          100.0
96          100.0
97          100.0
98          100.0
99          100.0

[100 rows x 1 columns]
       Unplanned (%)
count          100.0
mean           100.0
std              0.0
min            100.0
25%            100.0
50%            100.0
75%            100.0
max            100.0


In [ ]:
fig = px.scatter(stk_object.Planned_Data.sort_values(by="Target"),x="Time",y="Bin Number",hover_data="Bin Number",color="Satellite",animation_frame="Target")
fig.show()